## Practice 

In [257]:
import sys
import pandas as pd
import numpy as np
import streamlit as st
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
import plotly.graph_objects as go
import plotly.figure_factory as ff

#### Python Programming

In [258]:
#List and Tuple
my_list = [1,2,3,4,5]
my_tuple = (1,2,3,4,5)

print(f"List Size is {sys.getsizeof(my_list)}") # <-- list take more size
print(f"Tuple Size is {sys.getsizeof(my_tuple)}")

List Size is 104
Tuple Size is 80


In [259]:
#Dictionary

#Question 1
person = {"name": "Omkar", "city": "Bangalore", "job": "Intern"}
print(person['name'])
print(person['job'])

#Question 2 : .get method
person = {"name": "Omkar"}
# Get Salary if not in dictionary then print Not Available
print(person.get("salary", "Not Available"))

#Quesetion 3 : Nested Dictionary
data = {
    "country": "India",
    "stats": {
        "population": "1.4B",
        "rank": {"economy": 5, "cricket": 1}
    }
}
# Task: 'economy' rank (5) print karke dikhao.
print(data["stats"]['rank']['economy'])

Omkar
Intern
Not Available
5


In [260]:
#List Comprehension

#Question 1
names = ["omkar", "rahul", "amit"]
# Task: ["Omkar", "Rahul", "Amit"] banana hai
# Hint: .title() use karo

cap_name = [name.title() for name in names]
print(cap_name)


#Question 2
nums = [1, 2, 3, 4, 5, 6]

even_nums = [num for num in nums if num%2==0]
print(even_nums)


#Question 3
emails = ["omkar@gmail.com", "prof@iisc.ac.in", "student@iisc.ac.in", "spam@yahoo.com"]
iisc_emails = [email for email in emails if ('iisc.ac.in' in email)]
print(iisc_emails)


['Omkar', 'Rahul', 'Amit']
[2, 4, 6]
['prof@iisc.ac.in', 'student@iisc.ac.in']


In [261]:
# Lambda Function

#Question 1 find square
square = lambda x : x**2
square(2)

#Question 2 : add two numbers
addition = lambda x , y : x + y
addition(3,4)

#Question 3 : Convert into capital letter
df = pd.DataFrame({'City': ['bangalore', 'mumbai', 'delhi']})

df['City'].apply(lambda x : x.upper())

0    BANGALORE
1       MUMBAI
2        DELHI
Name: City, dtype: object

In [262]:
## Numpy Array

my_array = np.array(my_list)

# Convert list into array
print(my_array)
print(type(my_array))

# Vectorization of array : means operation without any loop
print(my_array * 5)

# Filter of array
print(my_array[my_array > 3])

[1 2 3 4 5]
<class 'numpy.ndarray'>
[ 5 10 15 20 25]
[4 5]


In [263]:
## Exception Handling

#Question 1
try:
    print(my_list[7]) 
except IndexError:
    print('Index is wrong')


#Question 2
try:
    print(int('Omkar'))
except ValueError as e:
    print(f'this is the error : {e}')



#Question 3
nums = [2,3,4,'omkar']
square = []

for num in nums:
    try:
        square.append(num**2)
    except TypeError:
        print(f"Skipping error : {num}")
print(square)

Index is wrong
this is the error : invalid literal for int() with base 10: 'Omkar'
Skipping error : omkar
[4, 9, 16]


In [264]:
# File Handling

# Write
with open('text.txt', 'w') as f:
    f.write('I am Omkar Sharma')

#Read
with open('text.txt', 'r') as f:
    content = f.read()
print(content)

#Append
with open('text.txt', 'a') as f:
    f.write('I am Universe')

I am Omkar Sharma


In [265]:
#Read line wise
with open('text.txt', 'r') as f:
    for line in f:
        print(line)

I am Omkar SharmaI am Universe


In [266]:
# Decorator in python
def decorator(func):
    print('Start function')
    func()
    print('End function')

@decorator
def my_func():
    print('Hello World!')

Start function
Hello World!
End function


In [267]:
@st.cache_data
def load_csv():
    df = pd.read_csv('data/cleaned_publications.csv')
    return df

2025-12-11 12:36:49.762 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [268]:
# Generator
def roti_maker():
    print("Roti 1 banayi")
    yield "Roti 1"  # Yahan ruk jayega
    
    print("Roti 2 banayi")
    yield "Roti 2"  # Agli baar yahan se chalega
    
    print("Roti 3 banayi")
    yield "Roti 3"

# Generator start kiya
my_gen = roti_maker()

# Ab maango
print(next(my_gen)) # Output: Roti 1 banayi -> Roti 1
print("--- Thodi der baad ---")
print(next(my_gen)) # Output: Roti 2 banayi -> Roti 2

Roti 1 banayi
Roti 1
--- Thodi der baad ---
Roti 2 banayi
Roti 2


#### Pandas Library

In [269]:
df = pd.read_csv('data/cleaned_publications.csv')
df.head()

,Country,Year,Documents,Times Cited,Documents in Top 1%,Documents in Top 10%,CNCI,Collab-CNCI,% Docs Cited,% Documents in Top 1%,% Documents in Top 10%,Rank,Benchmark Status
0,AUSTRALIA,2003,73479,3965411,1952,7645,1.39,1.33,96.88,1.76,13.41,1,Above Average (>= 1.0)
1,AUSTRALIA,2004,34122,4026396,561,2594,1.47,1.34,97.88,2.02,11.29,1,Above Average (>= 1.0)
2,AUSTRALIA,2005,76888,4458568,1177,8504,1.15,1.34,97.31,1.83,16.21,7,Above Average (>= 1.0)
3,AUSTRALIA,2006,69315,5190781,1369,6318,1.24,1.20,96.39,1.47,19.80,12,Above Average (>= 1.0)
4,AUSTRALIA,2007,11637,1443993,715,764,1.36,1.35,98.12,1.68,14.60,24,Above Average (>= 1.0)


In [270]:
#Selection and Filtering
df.loc[0:2, 'Documents':'CNCI'] 

df.iloc[0:2, 0:2]

df[df['Year'] > 2024]

,Country,Year,Documents,Times Cited,Documents in Top 1%,Documents in Top 10%,CNCI,Collab-CNCI,% Docs Cited,% Documents in Top 1%,% Documents in Top 10%,Rank,Benchmark Status
18,AUSTRALIA,2025,19731,868164,264,2623,1.00,0.84,99.51,1.61,22.23,2,Above Average (>= 1.0)
39,BRAZIL,2025,34221,2750341,437,4523,1.12,1.30,96.42,0.94,16.63,13,Above Average (>= 1.0)
61,CANADA,2025,19815,1325519,617,5893,1.39,1.21,97.70,1.31,22.61,2,Above Average (>= 1.0)
83,CHINA,2025,16475,1517035,814,1378,1.05,1.10,96.34,2.49,22.54,8,Above Average (>= 1.0)
104,FRANCE,2025,35819,2615938,657,3817,0.91,1.48,96.80,1.02,18.51,15,Below Average (< 1.0)
126,GERMANY,2025,79103,7506617,1085,6878,1.32,1.27,97.78,0.94,14.31,23,Above Average (>= 1.0)
147,INDIA,2025,52874,4288676,866,4204,1.44,1.29,97.13,1.15,22.86,6,Above Average (>= 1.0)
170,ITALY,2025,38944,2279097,1033,5068,1.49,1.25,96.32,1.48,23.09,2,Above Average (>= 1.0)
190,JAPAN,2025,64625,5780443,728,4089,1.32,1.26,97.39,2.04,13.40,3,Above Average (>= 1.0)
210,NETHERLANDS,2025,37062,3842733,878,5619,1.11,1.46,96.86,1.85,15.51,25,Above Average (>= 1.0)


In [271]:
#Handling NAN, Missing Value, Duplicates.
data = {
    'Country': ['India', 'USA', 'India', 'China', 'Japan', 'USA'],
    'Year': [2020, 2020, 2020, 2021, np.nan, 2020],  # Japan ka year gayab (NaN)
    'Documents': ['500', '1000', '500', '1500', '800', '1000'], # String format mein numbers
    'CNCI': [1.2, 1.5, 1.2, np.nan, 1.1, 1.5] # China ka CNCI gayab (NaN)
}

df_na = pd.DataFrame(data)
print("--- Raw Data ---")
display(df_na)

--- Raw Data ---


,Country,Year,Documents,CNCI
0,India,2020.0,500,1.2
1,USA,2020.0,1000,1.5
2,India,2020.0,500,1.2
3,China,2021.0,1500,NaN
4,Japan,NaN,800,1.1
5,USA,2020.0,1000,1.5


In [272]:
#Remove Duplicates
df_na = df_na.drop_duplicates(keep='first')
df_na

,Country,Year,Documents,CNCI
0,India,2020.0,500,1.2
1,USA,2020.0,1000,1.5
3,China,2021.0,1500,NaN
4,Japan,NaN,800,1.1


In [273]:
#Remove NA of Year
df_na = df_na.dropna(subset=['Year'])

df_na

,Country,Year,Documents,CNCI
0,India,2020.0,500,1.2
1,USA,2020.0,1000,1.5
3,China,2021.0,1500,NaN


In [274]:
#Fill NA of CNCI
mean_val = df['CNCI'].mean()

df_na = df_na.fillna({
    'CNCI' : mean_val
})

df_na

,Country,Year,Documents,CNCI
0,India,2020.0,500,1.200000
1,USA,2020.0,1000,1.500000
3,China,2021.0,1500,1.290412


In [275]:
# Convert datatypes 

print('Datatypes Before', df_na['Documents'].dtypes)

df_na['Documents'] = df_na['Documents'].astype('int')

print(df_na['Documents'].dtypes)

Datatypes Before object
int64


In [276]:
# Group by country total documents

df_doc = df.groupby('Country')['Documents'].sum().reset_index()

df_doc

,Country,Documents
0,AUSTRALIA,955154
1,BRAZIL,1086793
2,CANADA,976080
3,CHINA,942924
4,FRANCE,945200
5,GERMANY,789960
6,INDIA,849180
7,ITALY,926901
8,JAPAN,865418
9,NETHERLANDS,612548


In [277]:
# Group by country to find total documents and avg. cnci

df_doc_cnci = df.groupby('Country').agg({
    'Documents' : 'sum',
    'CNCI' : 'mean'
})

df_doc_cnci

,Documents,CNCI
Country,,
AUSTRALIA,955154,1.288421
BRAZIL,1086793,1.277619
CANADA,976080,1.281818
CHINA,942924,1.249545
FRANCE,945200,1.261429
GERMANY,789960,1.293636
INDIA,849180,1.280476
ITALY,926901,1.330000
JAPAN,865418,1.371500


In [278]:
# Create table where year is a column
df_year = df.pivot_table(index='Country', columns='Year', values='Documents', aggfunc='sum')

df_year

Year,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
Country,,,,,,,,,,,,,,,,,,,,,
AUSTRALIA,73479.0,34122.0,76888.0,69315.0,11637.0,124250.0,22569.0,39018.0,8580.0,64809.0,...,97265.0,NaN,22823.0,NaN,56223.0,73186.0,67310.0,1184.0,51246.0,19731.0
BRAZIL,NaN,27337.0,76379.0,25222.0,33470.0,30071.0,51225.0,29650.0,99292.0,119368.0,...,11503.0,60786.0,86514.0,96002.0,108043.0,66563.0,NaN,27014.0,37807.0,34221.0
CANADA,68007.0,104813.0,29760.0,108685.0,20819.0,68777.0,31800.0,7449.0,59636.0,NaN,...,35579.0,75948.0,4014.0,25011.0,40369.0,6040.0,24651.0,44001.0,51639.0,19815.0
CHINA,25164.0,67628.0,36386.0,16846.0,148329.0,28677.0,42772.0,41344.0,46022.0,21374.0,...,NaN,30185.0,36242.0,47126.0,85439.0,56368.0,40387.0,9524.0,36207.0,16475.0
FRANCE,78156.0,18398.0,46294.0,72233.0,56355.0,71974.0,NaN,7870.0,30010.0,15740.0,...,98020.0,37424.0,14098.0,44009.0,NaN,25107.0,52676.0,62252.0,32562.0,35819.0
GERMANY,65537.0,1569.0,14830.0,33170.0,69433.0,8565.0,NaN,20804.0,53658.0,25395.0,...,6858.0,37221.0,15098.0,53524.0,34432.0,30657.0,19481.0,14018.0,30944.0,79103.0
INDIA,45166.0,44837.0,NaN,13648.0,44838.0,65843.0,64558.0,58594.0,NaN,53599.0,...,74795.0,44457.0,19668.0,43548.0,62083.0,13997.0,7557.0,47342.0,27892.0,52874.0
ITALY,10695.0,149432.0,20076.0,34083.0,52487.0,54719.0,17614.0,73651.0,102937.0,47777.0,...,34429.0,10973.0,20069.0,15054.0,3503.0,29647.0,28800.0,56903.0,64545.0,38944.0
JAPAN,66978.0,18340.0,18772.0,40931.0,22919.0,13991.0,30973.0,NaN,101592.0,NaN,...,59395.0,80389.0,62743.0,51752.0,NaN,22500.0,5519.0,99461.0,2746.0,64625.0


In [279]:
# Concat and Merge
# Dataset 1: Research Output
df1 = pd.DataFrame({
    'Country': ['India', 'USA', 'China'],
    'Documents': [500, 1000, 1200]
})

# Dataset 2: Economic Data (Notice: China nahi hai, UK hai)
df2 = pd.DataFrame({
    'Country': ['India', 'USA', 'UK'],
    'GDP': ['3T', '23T', '3T']
})

# Dataset 3: New Data (Naya saal)
df3 = pd.DataFrame({
    'Country': ['Japan'],
    'Documents': [400]
})

print("--- Dataframes Created ---")

display(df1)
display(df2)
display(df3)

--- Dataframes Created ---


,Country,Documents
0,India,500
1,USA,1000
2,China,1200


,Country,GDP
0,India,3T
1,USA,23T
2,UK,3T


,Country,Documents
0,Japan,400


In [280]:
# Concatination

df_concat = pd.concat([df1, df3], ignore_index=True)

df_concat

,Country,Documents
0,India,500
1,USA,1000
2,China,1200
3,Japan,400


In [281]:
# Merge
df_inner_merge = pd.merge(df1, df2, on='Country', how='inner')

display(df_inner_merge)

df_left_merge = pd.merge(df1, df2, on='Country', how='left')

display(df_left_merge)


,Country,Documents,GDP
0,India,500,3T
1,USA,1000,23T


,Country,Documents,GDP
0,India,500,3T
1,USA,1000,23T
2,China,1200,NaN


In [282]:
# Vectorization
import pandas as pd
import numpy as np

# 1 Million Rows ka data banate hain
df_num = pd.DataFrame({
    'A': np.random.randint(1, 100, 1000000),
    'B': np.random.randint(1, 100, 1000000)
})


df_num

,A,B
0,56,60
1,26,94
2,97,59
3,5,8
4,62,45
...,...,...
999995,98,77
999996,39,14
999997,82,5
999998,72,79


In [283]:
#Vectorization
import time

# Tareeka 1: Vectorization (Fast)
start = time.time()
df['C'] = df['A'] + df['B']
print("Vectorization Time:", time.time() - start)

# Tareeka 2: Loop (Slow - Sirf dikhane ke liye, interview me mat karna)
# start = time.time()
# temp = []
# for i in range(len(df)):
#     temp.append(df['A'][i] + df['B'][i])
# print("Loop Time:", time.time() - start)

KeyError: 'A'

In [ ]:
# Ye code run tabhi hoga jab file ho, bas syntax samjho:

#Chunkization
# 1. Iterator banao
chunk_iterator = pd.read_csv('data/publications.csv', chunksize=1000)

total_sum = 0

# 2. Loop chalao (Generator logic)
for chunk in chunk_iterator:
    # 'chunk' ek chota dataframe hai (1000 rows ka)
    total_sum += chunk['Documents'].sum()

print("Grand Total:", total_sum)

KeyError: 'Documents'

##### Plotly Visualization

In [ ]:
# 2020 total document visualization
import plotly.express as px

df_2020 = df[df['Year'] == 2020]

bar_2020 = px.bar(
    df_2020,
    x='Documents',
    y='Country',
    title='2020 total doc.'
)

bar_2020.show()

In [ ]:
bar_group = px.bar(df, x='Year', y='Documents', color='Country', barmode='group')
bar_group.show()

In [ ]:
#go

# 1. Figure banao
fig = go.Figure()

# 2. Trace (Data) add karo
fig.add_trace(go.Bar(
    x=['India', 'USA', 'China'],
    y=[500, 2000, 2500],
    name='Documents', # Legend ka naam
    marker_color='indianred' # Custom color
))

# 3. Layout set karo
fig.update_layout(title="Manual Bar Chart")

fig.show()

In [ ]:
# 
fig = go.Figure()

# Layer 1: Bar Chart
fig.add_trace(go.Bar(
    x=['A', 'B', 'C'],
    y=[10, 20, 30],
    name='Sales'
))

# Layer 2: Line Chart (Isi ke upar)
fig.add_trace(go.Scatter(
    x=['A', 'B', 'C'],
    y=[15, 25, 5],
    name='Profit',
    mode='lines+markers' # Line aur dots dono
))

fig.show()

In [ ]:
fig = go.Figure(data=go.Bar(y=[2, 3, 1]))

# Magic Command: Pura structure print karo
print(fig)
# Output ko dhyan se dekho, tumhe 'data' aur 'layout' dikhega.

Figure({
    'data': [{'type': 'bar', 'y': [2, 3, 1]}], 'layout': {'template': '...'}
})


In [ ]:
fig = go.Figure(data=go.Bar(y=[2, 3, 1]))

# Layout dictionary ko update karna
fig.update_layout(
    title="My Fancy Chart",
    plot_bgcolor="lightgray", # Graph ka background
    xaxis_title="Categories"
)
fig.show()

In [ ]:
trace1 = go.Bar(y=[1, 2, 3], name="Sales")
trace2 = go.Scatter(y=[4, 5, 6], name="Target")

# Data ek LIST hai []
fig = go.Figure(data=[trace1, trace2]) 
fig.show()

In [ ]:
import plotly.express as px

df = px.data.tips() # Sample data
fig = px.bar(df, x="day", y="total_bill")

fig.update_layout(
    title_text="Weekly Sales Data",
    title_x=0.5,         # 0.5 matlab Center (0=Left, 1=Right)
    title_xanchor="center"
)

fig.update_layout(
    height=400,          # Height fix ki
    width=600,           # Width fix ki
    showlegend=True,
    legend=dict(
        orientation="h", # Horizontal (Leta hua)
        y=1.1,           # Graph ke upar
        x=0.5            # Center mein
    )
)

fig.update_layout(
    template="plotly_white", # Clean theme
    xaxis_title=None,        # X-axis label gayab
    yaxis_title="Revenue ($)" # Y-axis label change
)
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. Canvas banao (Do Y-Axis ke saath)
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 2. Pehla Data (Bar - Volume) -> Left Axis
fig.add_trace(
    go.Bar(x=['A', 'B', 'C'], y=[5000, 7000, 2000], name="Volume"),
    secondary_y=False  # Left side
)

# 3. Dusra Data (Line - Quality) -> Right Axis
fig.add_trace(
    go.Scatter(x=['A', 'B', 'C'], y=[1.2, 0.8, 1.5], name="Quality"),
    secondary_y=True   # Right side
)

# 4. Labeling
fig.update_yaxes(title_text="Volume (Thousands)", secondary_y=False)
fig.update_yaxes(title_text="Quality (Score)", secondary_y=True)

fig.show()

In [ ]:
import plotly.express as px
import pandas as pd

# Data
df = pd.DataFrame({
    'Country': ['India', 'USA', 'China'],
    'Volume': [500, 2000, 2500],
    'Quality': [1.2, 1.8, 0.9],
    'Rank': [10, 1, 2],       # Ye graph pe nahi dikhana, bas hover pe
    'Region': ['Asia', 'NA', 'Asia']
})

# Chart
fig = px.scatter(
    df, 
    x='Volume', 
    y='Quality',
    # Magic yahan hai:
    hover_name='Country',          # Bold mein naam aayega
    hover_data=['Rank', 'Region']  # Extra columns dikhenge
)

fig.show()

In [ ]:
df = pd.read_csv('data/cleaned_publications.csv')

df.head()

,Country,Year,Documents,Times Cited,Documents in Top 1%,Documents in Top 10%,CNCI,Collab-CNCI,% Docs Cited,% Documents in Top 1%,% Documents in Top 10%,Rank,Benchmark Status
0,AUSTRALIA,2003,73479,3965411,1952,7645,1.39,1.33,96.88,1.76,13.41,1,Above Average (>= 1.0)
1,AUSTRALIA,2004,34122,4026396,561,2594,1.47,1.34,97.88,2.02,11.29,1,Above Average (>= 1.0)
2,AUSTRALIA,2005,76888,4458568,1177,8504,1.15,1.34,97.31,1.83,16.21,7,Above Average (>= 1.0)
3,AUSTRALIA,2006,69315,5190781,1369,6318,1.24,1.20,96.39,1.47,19.80,12,Above Average (>= 1.0)
4,AUSTRALIA,2007,11637,1443993,715,764,1.36,1.35,98.12,1.68,14.60,24,Above Average (>= 1.0)


#### Q1 Top Performers Analysis: Quantity vs. Quality

**Objective:**
To identify the leading Countries in the dataset based on two distinct performance metrics:

1.  **Volume (Quantity):** Who is producing the most research?
    *   *Metric used:* `Documents`
2.  **Impact (Quality):** Who is producing the most influential research relative to their field?
    *   *Metric used:* `Category Normalized Citation Impact (CNCI)`

In [ ]:
# Configuration for aesthetic charts
df_doc = df.groupby('Country')['Documents'].sum().reset_index().sort_values(by='Documents', ascending=False).head()
df_cnci = df.groupby('Country')['CNCI'].mean().reset_index().sort_values(by='CNCI', ascending=False).head()


fig_doc = px.bar(
    df_doc,
    x='Documents',
    y='Country',
    orientation='h', # <-- horizontal
    text_auto='.2s',
    color='Documents',
    color_continuous_scale='Plasma',
    title='<b> Top 5 Countries by Research Documents Volume </b>', # <-- title 
    labels={                                                       # <-- label of chart
        'Documents' : 'Total Documents',
        'Country' : 'Country Name'
    }
)

fig_doc.update_layout(
    yaxis = dict(autorange='reversed'),
    coloraxis_showscale = False
)

fig_doc.show()

fig_cnci = px.bar(
    df_cnci,
    x='CNCI',
    y='Country',
    orientation='h', # <-- horizontal
    text_auto='.3f',
    color='CNCI',
    color_continuous_scale='plasma',
    title='<b> Top 5 Countries by Research Quality CNCI</b>', # <-- title
    labels={                                                  # <-- label of chart
        'CNCI' : 'Avg CNCI',
        'Country' : 'Country Name'
    }
)

fig_cnci.update_layout(
    yaxis = dict(autorange='reversed'),
    coloraxis_showscale = False
)

fig_cnci.add_vline(
    x=1.0,
    line_dash='dash',
    line_color='green',
    annotation_text = 'Global Avg(1.0)'
)

fig_cnci.show()


#### Q2 Ranges & Distributions Analysis: Research Relevance

**Objective:**
To analyze the spread and consistency of research relevance to understand how effectively published papers attract attention. This helps us determine if "getting cited" is a common standard or a rare achievement in this dataset.

1.  **Spread & Range:** What are the boundaries of performance?
    *   *Metric used:* `% Docs Cited` (Minimum vs. Maximum values).
2.  **Distribution Shape:** How is the performance distributed across the dataset?
    *   *Left Skewed:* Indicates most countries achieve high citation rates (Good sign).
    *   *Right Skewed:* Indicates most countries have low citation rates (Bad sign).
3.  **Consistency:** Is the citation rate stable across different countries and years, or are there massive disparities in performance?

In [ ]:
#Calculate Statistics
mean_value = df['% Docs Cited'].mean()
median_value = df['% Docs Cited'].median()

#Create plot of histogram
fig_dis = px.histogram(
    df,
    x='% Docs Cited',
    marginal='box',
    nbins=25,
    color_discrete_sequence=['gold'],
    opacity=0.5,
    title='Distribution of Research Relevence (% Docs Cited)',
    labels={
        '% Docs Cited' : 'Percentage of Document Cited'
    }
)

fig_dis.add_vline(x=mean_value, line_dash='dash', line_color='red')
fig_dis.add_vline(x=median_value, line_dash='dot', line_color='blue')


fig_dis.add_annotation(
    x=mean_value,
    y=1.15,
    yref='paper',
    showarrow=False,
    font=dict(color='red'),
    text=f"Mean : {mean_val:.1f}%"
)


fig_dis.add_annotation(
    x=median_value,
    y=1.08,
    yref='paper',
    showarrow=False,
    font=dict(color='blue'),
    text=f"Median : {median_value:.1f}%"
)


fig_dis.update_layout(
    bargap=0.1
)


fig_dis.show()

#### Q3. Benchmarks Analysis: Performance vs. Global Standards

**Objective:**
To evaluate national research performance against the established **Global Benchmark** to identify systemic strengths or weaknesses. This analysis helps distinguish between nations that consistently deliver "Elite" quality versus those that struggle to meet global expectations.

1.  **The Yardstick (Metric Definition):** How do we measure success?
    *   *Metric used:* `Category Normalized Citation Impact` (CNCI).
    *   **CNCI > 1.0:** Indicates research is performing **better** than the world average.
    *   **CNCI < 1.0:** Indicates research is performing **worse** than the world average.

2.  **Structural Strength (Aggregated View):** Are there any nations that are fundamentally weak?
    *   *Method:* Grouping data by `Country` to see if any nation's *average* performance falls below 1.0.

3.  **Consistency Check (Granular View):** How stable is the quality over time?
    *   *Method:* Analyzing individual year-wise data points to spot specific instances (anomalies) where a high-performing country might have had an "off-year" (CNCI < 1.0).

In [ ]:
#first create benchmark
df['Benchmark Status'] = df['CNCI'].apply(lambda x : 'Below CNCI 1.0' if x < 1.0 else 'Above CNCI 1.0') 


fig_strip = px.strip(
    df,
    x='Country',
    y='CNCI',
    color='Benchmark Status',
    color_discrete_map={
        'Below CNCI 1.0' : 'red',
        'Above CNCI 1.0' : 'gray'
    },
    hover_data=['Country', 'Year', 'CNCI'],
    template='plotly_white',
    title='Performance Distribution',
    labels={'CNCI' : 'CNCI (Impact)'}
)


fig_strip.add_hline(
    y=1.0,
    line_dash='dash',
    line_color='green',
    annotation_position='bottom right',
    annotation_text='Global Baseline(1.0)'
)

fig_strip.show()

#### Q4 Outlier Detection Analysis

**Objective:**
To identify data points that deviate significantly from the statistical norm, aiming to distinguish between potential data errors and genuine "Elite Performers" or "Mass Producers" that define the upper limits of research performance.

1.  **Volume Outliers ("Mass Producers"):** Which countries or years are producing a volume of documents that far exceeds the global distribution?
    *   *Metric used:* `Web of Science Documents` (Identifying Volume Extremes)
2.  **Impact Outliers ("Elite Performers"):** Are there specific instances (Country or Year) where the Citation Impact (CNCI) is exceptionally high or suspiciously low?
    *   *Metric used:* `Category Normalized Citation Impact (CNCI)` (Identifying Quality Anomalies)
3.  **Trend Consistency:** Do these outliers represent a consistent trend for a specific country, or are they isolated, one-off events?
    *   *Focus:* Analyzing the persistence of outliers across the `Year` column.

In [ ]:
doc_mean = df['Documents'].mean()

fig_docbox = px.box(
    df,
    y='Documents',
    points='all',
    color_discrete_sequence=['red'],
    hover_data=['Country', 'CNCI', 'Year'],
    title='Box Plot : Distribution of Documents',
    labels={
        'Documents' : 'Total Documents'
    },
    template='plotly_white',
    height=700
)

fig_docbox.add_hline(
    y=doc_mean,
    line_dash = 'dash',
    line_color = 'black',
    annotation_text = f'Mean Value {doc_mean:.1f}',
)

fig_docbox.show()



fig_cncibox = px.box(
    df,
    y='CNCI',
    points='all',
    color_discrete_sequence=['green'],
    hover_data=['Country', 'Documents', 'Year'],
    title='Box Plot : Distribution of CNCI',
    labels={
        'CNCI' : 'CNCI(Impact)'
    },
    template='plotly_white',
    height=700
)


fig_cncibox.add_hline(
    y=1.0,
    line_dash='dash',
    line_color='black',
    annotation_text='Global Baseline(1.0)'
)

fig_cncibox.show()

#### Q5 Funnel Check: Elite Research Conversion (% Documents in Top 1%)

**Objective:**
To evaluate the "elite research conversion rate" by quantifying the percentage of documents that achieve "Top 1%" status. This analysis benchmarks performance against the theoretical global average (1%) to identify nations that are not just good, but exceptionally impactful, and to understand the overall distribution of research excellence.

1.  **Benchmark Performance:** How does the dataset's average conversion rate compare to the theoretical global baseline of 1%?
    *   *Metric used:* Mean and Median of `% Documents in Top 1%`.
2.  **Distribution Shape:** What is the overall distribution of elite research performance? Is it symmetric, or skewed towards a few high-performing instances?
    *   *Focus:* Analyzing the Histogram and Box Plot for skewness and spread.
3.  **Identifying Consistent Leaders:** Which nations consistently surpass the "Exceptional Performance" threshold (>2%) across multiple years?
    *   *Focus:* Counting the frequency of each country's appearance in the >2% elite group.

In [ ]:
mean_top_val = df['% Documents in Top 1%'].mean()
median_top_val = df['% Documents in Top 1%'].median()

fig_top_dis = px.histogram(
    df,
    x='% Documents in Top 1%',
    marginal='box',
    color_discrete_sequence=['gold'],
    opacity=0.5,
    nbins=50,
    title='Distribution of Elite Output',
    labels={
        '% Documents in Top 1%' : 'Top 1% Elite Output'
    }
)

fig_top_dis.add_vline(
    x=mean_top_val,
    line_dash = 'dash',
    line_color = 'red',
)

fig_top_dis.add_vline(
    x=median_top_val,
    line_dash='dot',
    line_color='blue',
)

fig_top_dis.add_vline(
    x=1.0,
    line_dash='dash',
    line_color='black'
)

fig_top_dis.add_annotation(
    x=1.0,
    y=1.08,
    yref='paper',
    text='Global Baseline : 1.0%',
    showarrow=False,
    font=dict(color='black', size=10)
)

fig_top_dis.add_annotation(
    x=mean_top_val,
    y=1.15,
    yref='paper',
    text=f'Mean : {mean_top_val:.2f}%',
    showarrow=False,
    font = dict(color='red', size=10)
)

fig_top_dis.add_annotation(
    x=median_top_val,
    y=1.08,
    yref='paper',
    text=f'Median : {median_top_val:.2f}%',
    showarrow=False,
    font = dict(color='blue', size=10)
)

fig_top_dis.update_layout(
    bargap=0.1
)

fig_top_dis.show()

In [ ]:
df_top_country = df.groupby('Country')['% Documents in Top 1%'].mean().reset_index().sort_values(by='% Documents in Top 1%', ascending=False).head()

fig_top_bar = px.bar(
    df_top_country,
    x='% Documents in Top 1%',
    y='Country',
    color='% Documents in Top 1%',
    color_continuous_scale='plasma',
    text_auto= '.2f'
)

fig_top_bar.update_layout(
    yaxis = dict(autorange='reversed'),
    coloraxis_showscale = False
)

fig_top_bar.add_vline(
    x=1.0,
    line_dash = 'dash',
    line_color = 'black',
    annotation_text = 'Global Baseline : 1.0',
    annotation_position = 'top left'
)

fig_top_bar.show()

#### Q1 Top Performers Analysis: Publication Volume Trends

**Objective:**
To identify the **Top 5 countries** based on the *total volume of publications* in the dataset and then analyze their research output trends over the last two decades.

1.  **Identify Top Performers:** Who are the actual Top 5 countries based on the sum of `Web of Science Documents`?
2.  **Analyze Growth Trajectories:** How have these specific countries performed over time?

In [ ]:
top3_doc_count = df.groupby('Country')['Documents'].sum().reset_index().sort_values(by='Documents', ascending=False).head(3)

top3_doc_name = top3_doc_count['Country'].to_list()

top3_doc_count_trend = df[df['Country'].isin(top3_doc_name)].sort_values('Year')

top3_doc_count_trend

fig_line = px.line(
    top3_doc_count_trend,
    x='Year',
    y='Documents',
    color='Country',
    markers=True,
    template='plotly_white',
    title='<b>Top 3 Trends in Research Publications</b>',
    labels={
        'Year' : 'Year of Publications',
        'Documents' : 'Number of Documents Published'
    },
)

fig_line.update_layout(
        legend_title_text = '<b>Top 3 Countries</b>'
)

fig_line.show()

#### Q2: Quality vs. Time Trend

**Objective:**
To identify if there is a trade-off between the quantity of research (number of documents) and its quality (CNCI) over time, especially for major research-producing countries. We want to see if an increase in the number of publications leads to a decrease in the average quality.

1.  **How has the research quality (CNCI) changed over the years for major countries like Uk, Brazil and Spain?**
    *   **Metrics Used:** `Year`, `Category Normalized Citation Impact(CNCI)`, `Country`
2.  **Is there a visible inverse relationship when comparing the trend of `Web of Science Documents` and `CNCI` for these countries?**
    *   **Metrics Used:** `Year`, `Web of Science Documents`, `Category Normalized Citation Impact(CNCI)`, `Country`

In [ ]:
top5_doc_count = df.groupby('Country')['Documents'].sum().reset_index().sort_values(by='Documents', ascending=False).head(5)

top5_doc_name = top5_doc_count['Country'].to_list()

for country in top5_doc_name :
    top5_doc_count_trend = df[df['Country'] == country].sort_values('Year')
    fig_top_line = make_subplots(specs=[[{'secondary_y': True}]])

    fig_top_line.add_trace(
        go.Scatter(
            x=top5_doc_count_trend['Year'],
            y=top5_doc_count_trend['Documents'],
            name='Documents (Quantity)',
            mode='lines+markers',
            lines=dict(color='royalblue', width=3)
        ),
        secondary_y=False
    )

    fig_top_line.add_trace(
        go.Scatter(
            x=top5_doc_count_trend['Year'],
            y=top5_doc_count_trend['CNCI'],
            name='CNCI (Quality)',
            mode='lines+markers',
            lines=dict(color='firebrick', width=3, dash='dash')
        ),
        secondary_y=True,
    )

    fig_top_line.show()

ValueError: Invalid property specified for object of type plotly.graph_objs.Scatter: 'lines'

Did you mean "line"?

    Valid properties:
        alignmentgroup
            Set several traces linked to the same position axis or
            matching axes to the same alignmentgroup. This controls
            whether bars compute their positional range dependently
            or independently.
        cliponaxis
            Determines whether or not markers and text nodes are
            clipped about the subplot axes. To show markers and
            text nodes above axis lines and tick labels, make sure
            to set `xaxis.layer` and `yaxis.layer` to *below
            traces*.
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
         

#### Q3: Collaboration vs. Overall Quality

**Objective:**
To identify the relationship between the impact of collaborative research and the overall research quality across different countries and years.

1.  **Is there a positive correlation between `Collab-CNCI` and `CNCI`?**
    *   **Metrics Used:** `Collab-CNCI`, `Category Normalized Citation Impact (CNCI)`
2.  **Does higher collaboration impact consistently lead to higher overall quality?**
    *   **Metrics Used:** Visual inspection of the scatter plot trend.

In [ ]:
correlation = df['Collab-CNCI'].corr(df['CNCI'])

print(f'the correlation between Collab-CNCI and CNCI is {round(correlation,2)}')


fig_corr = px.scatter(
    df,
    x='Collab-CNCI',
    y='CNCI',
    trendline='ols',
    hover_data=['Country', 'Year'],
    template='plotly_white'
)

fig_corr.update_traces(
    selector=dict(mode='lines'),
    line= dict(color='red', width=3)
)

fig_corr.show()

the correlation between Collab-CNCI and CNCI is -0.07


#### Q4: Rank vs. Performance Metrics

**Objective:**
To determine whether a country's `Rank` in this dataset is more strongly influenced by the sheer `quantity` of its research (`Web of Science Documents`) or the overall `impact` of that research (`Times Cited`).

1.  **How does Rank correlate with the number of documents?**
    *   **Metrices used:** `Rank`, `Web of Science Documents`
2.  **How does Rank correlate with the number of times cited?**
    *   **Metrices used:** `Rank`, `Times Cited`

In [ ]:
corr_columns = ['CNCI', 'Rank', 'Documents', 'Times Cited']

df_corr = df[corr_columns].corr()

df_corr


fig_heatmap = px.imshow(
    df_corr,
    text_auto = True,
    aspect='auto',
    color_continuous_scale='Oranges'
)

fig_heatmap.show()

#### Q5. Gap Analysis: Rank 1 vs Rank 2 (Times Cited)

**Objective:**
To measure the dominance of the market leader. Since the provided `Rank` column has weak correlation and duplicates, we will dynamically identify the **True Top 2 Performers** based purely on `Times Cited` for each year and analyze the gap between them.

**1. Who are the actual Top 2 players by Impact?**
*   **Metrics used:** `Country`, `Times Cited` (Sorted Descending per Year).

**2. How big is the dominance gap?**
*   **Metrics used:**
    *   **Absolute Gap:** (Leader Citations - Runner-up Citations).
    *   **Dominance %:** How much larger (%) is the Leader compared to the Runner-up.

In [ ]:
gap_data = []

unique_years = df['Year'].unique()

unique_years


for year in unique_years:
    df_timescited = df[df['Year'] == year].sort_values(by='Times Cited', ascending=False)

    if len(df_timescited) > 2:
        leader = df_timescited.iloc[0]
        runner_up = df_timescited.iloc[1]

        abs_gap = leader['Times Cited'] - runner_up['Times Cited']

        if runner_up['Times Cited'] > 0:
            pct_gap = abs_gap*100 / runner_up['Times Cited']
        else:
            pct_gap = 0
        
        gap_data.append({
            'Year' : year,
            'Leader Name' : leader['Country'],
            'Runnerup Name' : runner_up['Country'],
            'Leader Citation' : leader['Times Cited'],
            'Runnerup Citation' : runner_up['Times Cited'],
            'Absolute Gap' : abs_gap,
            'Dominance %' : pct_gap
         })
        

df_dominance = pd.DataFrame(gap_data)

df_dominance

fig_dominance = go.Figure()

fig_dominance.add_trace(
    go.Bar(
        x=df_dominance['Year'],
        y=df_dominance['Leader Citation'],
        marker_color = 'red',
        text=df_dominance['Leader Name']

    )
)

fig_dominance.add_trace(
    go.Bar(
        x=df_dominance['Year'],
        y=df_dominance['Runnerup Citation'],
        marker_color = 'blue',
        text=df_dominance['Runnerup Name']
    )
)


fig_dominance.update_layout(
    template='plotly_white'
)

fig_dominance.show()


fig_line_dominance = px.line(
    df_dominance,
    x='Year',
    y='Dominance %',
    markers=True,
    template='plotly_white',
    hover_data=['Leader Name', 'Runnerup Name', 'Dominance %']
)

fig_line_dominance.update_traces(
    line_color='crimson',
    marker=dict(size=10)
)

fig_line_dominance.show()

#### Q1. Segmentation Analysis: Quantity vs. Quality (Quadrants)

**Objective:**
To classify countries/entities into strategic performance groups based on their volume of work and the impact of that work. This helps in identifying who is driving global research quality versus who is merely increasing volume.

**1. Who are the "Elite Players"?**
*   **Definition:** Countries producing a high volume of papers with high citation impact.
*   **Metrics used:** High `Web of Science Documents` (> Median) **AND** High `CNCI` (> Median).

**2. Who are the "Mass Producers"?**
*   **Definition:** Countries publishing a lot of papers but with below-average impact (Quantity over Quality).
*   **Metrics used:** High `Web of Science Documents` (> Median) **BUT** Low `CNCI` (< Median).

**3. Who are the "Niche / High Potential" players?**
*   **Definition:** Smaller countries producing fewer papers, but of exceptional quality.
*   **Metrics used:** Low `Web of Science Documents` (< Median) **BUT** High `CNCI` (> Median).

**4. Who are the "Underperformers"?**
*   **Definition:** Countries struggling with both volume and impact.
*   **Metrics used:** Low `Web of Science Documents` (< Median) **AND** Low `CNCI` (< Median).

In [ ]:
overall_df = df.groupby('Country').agg({
    'Documents' : 'sum',
    'Times Cited' : 'sum',
    'CNCI' : 'mean',
    'Collab-CNCI' : 'mean',
    '% Docs Cited' : 'mean'
}).reset_index()

cnci_med_line = overall_df['CNCI'].median()
doc_med_line = overall_df['Documents'].median()

fig_scatter = px.scatter(
    overall_df,
    x='Documents',
    y='CNCI',
    color='Collab-CNCI',
    size='Times Cited',
    color_continuous_scale='plasma'
)

fig_scatter.add_vline(
    x=doc_med_line,
    line_dash='dash',
    line_color='red',
    annotation_text=f'Median : {doc_med_line:.2f}'
)

fig_scatter.add_hline(
    y=cnci_med_line,
    line_dash='dash',
    line_color='red',
    annotation_text = f'Median : {cnci_med_line: .2f}%'
)

fig_scatter.update_layout(
    template='plotly_white'
)

fig_scatter.show()

#### Q2: Pareto Principle (80/20 Rule) Analysis

**Objective:**
To identify the concentration of research impact and determine if a small minority of entities (e.g., top countries) are responsible for the majority of the global citations (influence). We want to validate if the "Vital Few" drive the results.

**[Question 1]**
Do the Top 20% of countries generate approx. 80% of the Total Citations?
*   **Metrics used:** `Country`, `Times Cited` (sorted), `Cumulative Percentage of Citations`.

**[Question 2]**
Which specific countries fall into this "Vital Few" (Top contributors) category?
*   **Metrics used:** `Country`, `Times Cited`, `Rank` (based on contribution).

In [ ]:
pareto_df = df.groupby('Country')['Times Cited'].sum().reset_index().sort_values(by='Times Cited', ascending=False)

total_citation = pareto_df['Times Cited'].sum()

pareto_df['Commulative Citation'] = pareto_df['Times Cited'].cumsum()
pareto_df['Commulative pct'] = pareto_df['Commulative Citation'] / total_citation


fig_pareto = make_subplots(specs=[[{'secondary_y' : True}]])

fig_pareto.add_trace(
    go.Bar(
        x=pareto_df['Country'],
        y=pareto_df['Times Cited'],
        name='Citation (Volume)',
        marker_color='red',

    ),
    secondary_y=False,
)

fig_pareto.add_trace(
    go.Scatter(
        x=pareto_df['Country'],
        y=pareto_df['Commulative pct'],
        name='Commulative %',
        marker=dict(size=6, color='blue'),
        line=dict(width=3),

    ),
    secondary_y=True
)

fig_pareto.show()

#### Q3. Elite Conversion: Collaboration Impact on Top 1% Papers

**Objective:**
To identify if high-quality collaborations (`Collab-CNCI`) directly translate into elite-level research output (producing papers in the global Top 1%). We want to see if working effectively with others helps produce "Blockbuster" research.

**Key Questions & Metrics:**

1.  **Correlation Check:** Is there a strong positive relationship between Collaboration Quality and Elite Output?
    *   **Metrics Used:** `Collab-CNCI` (X-axis) vs. `% Documents in Top 1%` (Y-axis).
2.  **Identifying Super-Collaborators:** Which countries/entities maintain high collaboration standards AND produce a high volume of elite papers?
    *   **Metrics Used:** `Country` (Label), `Web of Science Documents` (Bubble Size), `Category Normalized Citation Impact` (Color).

In [285]:
# --- Step-1 : Data Aggregation (Country-Level Strategy View) ---
df_agg = df.groupby('Country').agg({
    'Collab-CNCI': 'mean',                  # <-- for X-Axis: Avg Collaboration Quality
    '% Documents in Top 1%': 'mean',        # <-- for Y-Axis: Avg Elite Output
    'Documents': 'sum',                     # <-- for Bubble Size: Total Volume
    'CNCI': 'mean'                          # <-- for Color: Overall Quality
}).reset_index()

# Calculate Correlation
correlation = df_agg['Collab-CNCI'].corr(df_agg['% Documents in Top 1%'])
print(f"Pearson Correlation Coefficient: {correlation:.4f}")

# --- Step-2 : Create Advanced Bubble Chart
fig = px.scatter(
    df_agg,
    x="Collab-CNCI",
    y="% Documents in Top 1%",
    size="Documents",                       # <-- give Volume as a Bubble Size
    color="CNCI",                           # <-- give quality(CNCI) as a color
    hover_name="Country",                   
    size_max=60,                          
    color_continuous_scale="Viridis",       # <-- give continuous color
    title=f"<b>Elite Conversion Analysis: Collaboration vs. Top 1% Papers</b><br><sup>Correlation: {correlation:.2f} (Size = Volume, Color = Overall CNCI)</sup>",
    template="plotly_white",
    trendline="ols"                         # <-- ordinary Least Squares
)

# Customizing the Layout for Better Insights
fig.update_traces(
    marker=dict(line=dict(width=1, color='DarkSlateGrey')) # <-- create border on bubble
)

# bold and red trendline
fig.update_traces(selector=dict(mode='lines'), line=dict(color='red', width=3, dash='solid'))

avg_collab = df_agg['Collab-CNCI'].mean()
avg_elite = df_agg['% Documents in Top 1%'].mean()

fig.add_hline(y=avg_elite, line_dash="dot", annotation_text=f"Avg Elite Output ({avg_elite:.2f}%)", annotation_position="bottom right")
fig.add_vline(x=avg_collab, line_dash="dot", annotation_text=f"Avg Collab Quality ({avg_collab:.2f})", annotation_position="top right")

# Axis Titles
fig.update_layout(
    xaxis_title="Collaboration Quality (Collab-CNCI)",
    yaxis_title="Elite Research Output (% in Top 1%)",
    height=700,
    width=1000,
    font=dict(size=12)
)

fig.show()

Pearson Correlation Coefficient: -0.0186
